## 3.2.2 BM25 Score 구하기 코드 실습

In [10]:
# 설치
!pip install rank_bm25
!pip install pandas
!pip install konlpy


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from rank_bm25 import BM25Okapi
import pandas as pd
import konlpy

In [12]:
# 문서 정의
documents = [
    "고양이와 강아지", # 문서1
    "우리 강아지가 다른 강아지를 쫓아갔어", # 문서2
    "강아지는 창밖을 봐" # 문서3 
]

In [ ]:

# Okt 형태소 분석기 초기화
okt = konlpy.tag.Okt()

# 문서 내용 토큰화 함수 정의
def tokenize_korean_text(text):
    return okt.morphs(text) #


for doc in documents:
    print(tokenize_korean_text(doc))

['고양이', '와', '강아지']
['우리', '강아지', '가', '다른', '강아지', '를', '쫓아갔어']
['강아지', '는', '창밖', '을', '봐']


In [ ]:
# 각 문서를 토큰화 (단어 단위로 분할)
tokenize_documents=[]
for doc in documents:
    tokenize_documents.append(tokenize_korean_text(doc))
print(tokenize_documents)

[['고양이', '와', '강아지'], ['우리', '강아지', '가', '다른', '강아지', '를', '쫓아갔어'], ['강아지', '는', '창밖', '을', '봐']]


In [ ]:
# particles 리스트에 있는 불용어를 제거하는 함수
def delete_particles(torkenized_docs:list,
                     particles:list=["와", "가", "를", "는", "을"]):
    for sentence_list in torkenized_docs:
        for token in sentence_list:
            if token in particles:
                sentence_list.remove(token)
    return torkenized_docs

print(delete_particles(torkenized_docs=tokenize_documents))

[['고양이', '강아지'], ['우리', '강아지', '다른', '강아지', '쫓아갔어'], ['강아지', '창밖', '봐']]


In [32]:
# BM25 객체 생성
bm25 = BM25Okapi(tokenized_result, k1=1.5, b=0.75)

In [33]:
# 특정 단어들에 대해 각 문서에서의 BM25 점수 확인
terms = list(set([word for doc in tokenized_result for word in doc]))  # 문서의 모든 단어를 추출하여 고유한 단어 목록 생성
terms

['봐', '고양이', '쫓아갔어', '강아지', '다른', '창밖', '우리']

In [34]:
# 각 문서에 대해 BM25 점수를 계산하여 저장
bm25_scores=[]
for term in terms:
    term_scores = bm25.get_scores([term])
    bm25_scores.append(term_scores)
bm25_scores


[array([0.        , 0.        , 0.53489594]),
 array([0.62295808, 0.        , 0.        ]),
 array([0.        , 0.41700051, 0.        ]),
 array([0.04873883, 0.04918873, 0.04184905]),
 array([0.        , 0.41700051, 0.        ]),
 array([0.        , 0.        , 0.53489594]),
 array([0.        , 0.41700051, 0.        ])]

In [35]:
# 결과를 데이터프레임으로 변환
bm25_df = pd.DataFrame(bm25_scores, index=terms, columns=['문서1', '문서2', '문서3'])

In [36]:
# 결과 출력
print("BM25 Scores Matrix:")
print(bm25_df)

BM25 Scores Matrix:
           문서1       문서2       문서3
봐     0.000000  0.000000  0.534896
고양이   0.622958  0.000000  0.000000
쫓아갔어  0.000000  0.417001  0.000000
강아지   0.048739  0.049189  0.041849
다른    0.000000  0.417001  0.000000
창밖    0.000000  0.000000  0.534896
우리    0.000000  0.417001  0.000000
